<a href="https://colab.research.google.com/github/alexcassis/movie_recomendations/blob/master/Evaluation3_prog2_alexcassis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installation de PySpark (N'oubliez pas de charger le fichier AAPL.csv)

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 18.0 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=8126acadde421ec9238c31875e1d785f9a344305161fdd767eb23dcb7347be89
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


Création de la session Spark et importation du fichier AAPL.csv qui a été téléchargé dans le répertoire Colab.

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder \
    .appName("Analyse transactions en bourse pour AAPL") \
    .master("local[*]") \
    .getOrCreate()

aaplStockData = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("AAPL.csv")

aaplStockData.createOrReplaceTempView("aapl_csv")


 Liste des dates de transaction, ouverture et fermeture de la valeur de l’action

In [3]:
query1 = spark.sql("SELECT Date, Open, Close FROM aapl_csv")
query1.show()

+-------------------+----------+----------+
|               Date|      Open|     Close|
+-------------------+----------+----------+
|2014-09-29 00:00:00|100.589996| 99.620003|
|2014-10-06 00:00:00| 99.949997|100.730003|
|2014-10-13 00:00:00|101.330002| 97.669998|
|2014-10-20 00:00:00|     98.32|105.220001|
|2014-10-27 00:00:00|104.849998|     108.0|
|2014-11-03 00:00:00|108.220001|109.010002|
|2014-11-10 00:00:00|109.019997|    114.18|
|2014-11-17 00:00:00|114.269997|116.470001|
|2014-11-24 00:00:00|116.849998|    118.93|
|2014-12-01 00:00:00|118.809998|     115.0|
|2014-12-08 00:00:00|114.099998|109.730003|
|2014-12-15 00:00:00|110.699997|111.779999|
|2014-12-22 00:00:00|112.160004|113.989998|
|2014-12-29 00:00:00|113.790001|109.330002|
|2015-01-05 00:00:00|108.290001|112.010002|
|2015-01-12 00:00:00|112.599998|105.989998|
|2015-01-19 00:00:00|107.839996|112.980003|
|2015-01-26 00:00:00|113.739998|117.160004|
|2015-02-02 00:00:00|118.050003|    118.93|
|2015-02-09 00:00:00|118.550003|

Liste des dates de transaction avec la différence entre fermeture et ouverture

In [4]:
query2 = spark.sql("SELECT Date, (Close - Open) AS Difference FROM aapl_csv")
query2.show()

+-------------------+-------------------+
|               Date|         Difference|
+-------------------+-------------------+
|2014-09-29 00:00:00|-0.9699930000000023|
|2014-10-06 00:00:00| 0.7800060000000002|
|2014-10-13 00:00:00|-3.6600039999999865|
|2014-10-20 00:00:00|  6.900001000000003|
|2014-10-27 00:00:00| 3.1500020000000006|
|2014-11-03 00:00:00| 0.7900010000000037|
|2014-11-10 00:00:00|  5.160003000000003|
|2014-11-17 00:00:00| 2.2000039999999927|
|2014-11-24 00:00:00| 2.0800020000000075|
|2014-12-01 00:00:00| -3.809997999999993|
|2014-12-08 00:00:00| -4.369995000000003|
|2014-12-15 00:00:00| 1.0800020000000075|
|2014-12-22 00:00:00| 1.8299939999999992|
|2014-12-29 00:00:00|-4.4599990000000105|
|2015-01-05 00:00:00| 3.7200009999999963|
|2015-01-12 00:00:00| -6.609999999999999|
|2015-01-19 00:00:00|  5.140006999999997|
|2015-01-26 00:00:00| 3.4200060000000008|
|2015-02-02 00:00:00|  0.879997000000003|
|2015-02-09 00:00:00|   8.52999899999999|
+-------------------+-------------

Max et Min des volumes

In [5]:
query3 = spark.sql("SELECT MAX(Volume) AS MaxVolume, MIN(Volume) AS MinVolume FROM aapl_csv")
query3.show()

+---------+---------+
|MaxVolume|MinVolume|
+---------+---------+
|500363000| 38398505|
+---------+---------+



Moyenne des valeurs d’ouverture par années

In [7]:
query4 = spark.sql("""
  SELECT YEAR(Date) AS Year, AVG(Open) AS AvgOpen
  FROM aapl_csv
  GROUP BY YEAR(Date)
  ORDER BY YEAR(Date)
""")
query4.show()

+----+------------------+
|Year|           AvgOpen|
+----+------------------+
|2014|108.78285600000001|
|2015|120.08519211538461|
|2016|104.27115398076923|
|2017|149.64134611538464|
|2018|171.75464521428572|
+----+------------------+



Somme des volumes par mois

In [6]:
query5 = spark.sql("""
  SELECT YEAR(Date) AS Year, MONTH(Date) AS Month, SUM(Volume) AS TotalVolume
  FROM aapl_csv
  GROUP BY YEAR(Date), MONTH(Date)
  ORDER BY YEAR(Date), MONTH(Date)
""")
query5.show()

+----+-----+-----------+
|Year|Month|TotalVolume|
+----+-----+-----------+
|2014|    9|  142718700|
|2014|   10| 1217561500|
|2014|   11|  820408200|
|2014|   12| 1126799200|
|2015|    1| 1251439200|
|2015|    2| 1136535200|
|2015|    3| 1211483600|
|2015|    4|  981806600|
|2015|    5|  895639500|
|2015|    6|  935962400|
|2015|    7| 1000057400|
|2015|    8| 1847943200|
|2015|    9| 1086531300|
|2015|   10|  990374200|
|2015|   11|  918226100|
|2015|   12|  753944200|
|2016|    1| 1271024100|
|2016|    2|  977450100|
|2016|    3|  605336100|
|2016|    4|  846522200|
+----+-----+-----------+
only showing top 20 rows



Calcul de la moyenne mobile sur 10 jours

In [8]:
from pyspark.sql.window import Window

# Calculer la moyenne mobile sur 10 jours
windowSpec = Window.orderBy("Date").rowsBetween(-9, 0)
aaplStockDataWithMovingAverage = aaplStockData \
    .withColumn("10_day_MA", avg("Close").over(windowSpec))

# Montrer les 20 premiers résultats
aaplStockDataWithMovingAverage.select("Date", "Close", "10_day_MA").show(20)


+-------------------+----------+------------------+
|               Date|     Close|         10_day_MA|
+-------------------+----------+------------------+
|2014-09-29 00:00:00| 99.620003|         99.620003|
|2014-10-06 00:00:00|100.730003|        100.175003|
|2014-10-13 00:00:00| 97.669998| 99.34000133333335|
|2014-10-20 00:00:00|105.220001|      100.81000125|
|2014-10-27 00:00:00|     108.0|        102.248001|
|2014-11-03 00:00:00|109.010002|103.37500116666666|
|2014-11-10 00:00:00|    114.18|104.91857242857142|
|2014-11-17 00:00:00|116.470001|        106.362501|
|2014-11-24 00:00:00|    118.93|107.75888977777777|
|2014-12-01 00:00:00|     115.0|108.48300079999999|
|2014-12-08 00:00:00|109.730003|109.49400079999998|
|2014-12-15 00:00:00|111.779999|       110.5990004|
|2014-12-22 00:00:00|113.989998|       112.2310004|
|2014-12-29 00:00:00|109.330002|       112.6420005|
|2015-01-05 00:00:00|112.010002|113.04300070000002|
|2015-01-12 00:00:00|105.989998|       112.7410003|
|2015-01-19 